In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 893.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall -y trl unsloth unsloth-zoo peft transformers bitsandbytes
!pip install -q datasets evaluate rouge-score bert-score

Found existing installation: trl 0.24.0
Uninstalling trl-0.24.0:
  Successfully uninstalled trl-0.24.0
Found existing installation: unsloth 2026.1.2
Uninstalling unsloth-2026.1.2:
  Successfully uninstalled unsloth-2026.1.2
Found existing installation: unsloth_zoo 2026.1.2
Uninstalling unsloth_zoo-2026.1.2:
  Successfully uninstalled unsloth_zoo-2026.1.2
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 151.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [ ]:
!pip install torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.9.0+cu126
    Uninstalling torchaudio-2.9.0+cu126:
      Successfully uninstalled torchaudio-2.9.0+cu126


In [ ]:
!pip install -q datasets evaluate rouge-score bert-score



In [ ]:
import torch, random, numpy as np
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
import evaluate

/tmp/ipython-input-389265459.py:3: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from datasets import load_dataset

raw = load_dataset("cnn_dailymail", "3.0.0")

def count_sentences(text):
    return len(text.split(". "))

def format_example(example):
    n = count_sentences(example["highlights"])

    if n <= 2:
        length = "short"
        instruction = "Summarize the article in 2-3 sentences."
    elif n <= 4:
        length = "medium"
        instruction = "Summarize the article in 3-4 sentences."
    else:
        length = "long"
        instruction = "Summarize the article in 5-6 sentences."

    return {
        "text": f"""### Instruction:
{instruction}

### Article:
{example['article'][:3000]}

### Summary:
{example['highlights']}
"""
    }

train_dataset = (
    raw["train"]
    .shuffle(seed=42)
    .select(range(1000))
    .map(format_example, remove_columns=raw["train"].column_names)
)

val_dataset = (
    raw["validation"]
    .shuffle(seed=42)
    .select(range(200))
    .map(format_example, remove_columns=raw["validation"].column_names)
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length = 1024,
    dtype = torch.float16,
    load_in_4bit = True,
)

==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.0,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    num_train_epochs = 2,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    save_strategy = "epoch",
    optim = "adamw_8bit",
    report_to = "none",
    seed = 42,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = 1024,
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(format_example(raw["train"][0])["text"])

### Instruction:
Summarize the article in 2-3 sentences.

### Article:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six place

In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 126
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Step,Training Loss


In [ ]:
adapter_path = "/content/mistral-slm-summary"
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)


In [ ]:
import shutil

zip_path = "/content/mistral-slm-summary.zip"
shutil.make_archive(
    base_name=zip_path.replace(".zip", ""),
    format="zip",
    root_dir=adapter_path
)

zip_path


In [ ]:
from google.colab import files
files.download(zip_path)

In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer
from bert_score import score as bertscore
import matplotlib.pyplot as plt

In [ ]:
import torch
from datasets import load_dataset
from rouge_score import rouge_scorer
from bert_score import score # Explicitly import score function for BERTScore
from transformers import AutoTokenizer, GenerationConfig, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM # Import AutoPeftModelForCausalLM
from unsloth import FastLanguageModel # Import FastLanguageModel (needed for patching)


# --------------------------------------------
# Config
# --------------------------------------------
MODEL_DIR = "/content/mistral-slm-summary"
EXAMPLE_ID = 25          # any index you like
SUMMARY_LENGTH = "short"

# --------------------------------------------
# Load model & tokenizer
# --------------------------------------------

# Initialize quantization config (not directly used by AutoPeftModelForCausalLM when loading from a directory)
# AutoPeftModelForCausalLM loads the quantization config from the saved model.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the PEFT model with its base model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map={"": 0},   # 👈 force everything to GPU 0
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

tokenizer.pad_token = tokenizer.eos_token
model.eval()
model.config.use_cache = True # Enable for inference

# --------------------------------------------
# Generation config
# --------------------------------------------
GEN_CONFIG = GenerationConfig(
    max_new_tokens=90,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)

# --------------------------------------------
# Summarization function
# --------------------------------------------
def summarize(article, length):
    if length == "short":
        instruction = "Summarize the article in 2-3 sentences.";
        max_tokens = 80;
    elif length == "medium":
        instruction = "Summarize the article in 3 sentences.";
        max_tokens = 120;
    else:
        instruction = "Summarize the article in 5 sentences.";
        max_tokens = 180;

    prompt = f"""### Instruction:
{instruction}

### Article:
{article}

### Summary:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            repetition_penalty=1.3,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("### Summary:")[-1].strip()

# --------------------------------------------
# Load one dataset example
# --------------------------------------------
dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation")

article = dataset[EXAMPLE_ID]["article"]
reference = dataset[EXAMPLE_ID]["highlights"]

# --------------------------------------------
# Run inference
# --------------------------------------------
prediction = summarize(article, SUMMARY_LENGTH)

# --------------------------------------------
# Print sanity check outputs
# --------------------------------------------
print("=" * 100)
print("ARTICLE (truncated):\n")
print(article[:900] + "...\n")
print("-" * 100)
print("REFERENCE SUMMARY:\n")
print(reference + "\n")
print("-" * 100)
print("MODEL SUMMARY:\n")
print(prediction)
print("=" * 100)

# --------------------------------------------
# ROUGE evaluation (single example)
# --------------------------------------------
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeLsum"],
    use_stemmer=True
)

rouge_scores = scorer.score(reference, prediction)

print("\nROUGE SCORES (single example):")
for k, v in rouge_scores.items():
    print(f"{k.upper():10s} | P: {v.precision:.4f} | R: {v.recall:.4f} | F1: {v.fmeasure:.4f}")

# --------------------------------------------
# BERTScore evaluation
# --------------------------------------------
P, R, F1 = score(
    [prediction],
    [reference],
    lang="en",
    verbose=False
)

print("\nBERTScore:")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")

/tmp/ipython-input-3838431439.py:7: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel # Import FastLanguageModel (needed for patching)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


`torch_dtype` is deprecated! Use `dtype` instead!


ARTICLE (truncated):

(Rolling Stone)Kanye West fans are used to seeing thousand-dollar price tags attached to the rapper's Air Yeezy sneakers when they're put up for sale on eBay, but the auction site is now being overrun by sellers offering up plastic bags full of air allegedly captured at West's concerts. The gag started Friday when one ambitious seller attempted to sell a Zipperseal bag with "Air From Kanye Show," with 90 bidders driving the price tag to over $60,000, the Telegraph reports. While that auction was ultimately pulled from the auction site, eBay has since been flooded with similar auctions promising air bagged from a Yeezus show, even though the copycats lacked certificates of authenticity proving the air -- likely carbon dioxide -- was grabbed at a West concert. eBay has apparently stopped trying to swat down the new "bagged air" auctions, so buyer beware at this point. Rolling Stone: Kanye ...

-------------------------------------------------------------------------

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore:
Precision: 0.8443
Recall:    0.8677
F1:        0.8558


In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation")

N_SAMPLES = 50
indices = list(range(N_SAMPLES))


NameError: name 'load_dataset' is not defined

In [ ]:
rouge1_f1, rouge2_f1, rougel_f1 = [], [], []
bert_f1 = []

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeLsum"], use_stemmer=True
)

for idx in indices:
    article = dataset[idx]["article"][:3000]
    reference = dataset[idx]["highlights"]

    prompt = f"""### Instruction:
Summarize the article below in a short summary (3 sentences max).

### Article:
{article}

### Summary:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = decoded.split("### Summary:")[-1].strip()

    rouge = scorer.score(reference, prediction)
    rouge1_f1.append(rouge["rouge1"].fmeasure)
    rouge2_f1.append(rouge["rouge2"].fmeasure)
    rougel_f1.append(rouge["rougeLsum"].fmeasure)

    _, _, F1 = bertscore([prediction], [reference], lang="en", verbose=False)
    bert_f1.append(F1.mean().item())


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("===== AVERAGE METRICS =====")
print(f"ROUGE-1 F1:   {np.mean(rouge1_f1):.4f}")
print(f"ROUGE-2 F1:   {np.mean(rouge2_f1):.4f}")
print(f"ROUGE-L F1:   {np.mean(rougel_f1):.4f}")
print(f"BERTScore F1:{np.mean(bert_f1):.4f}")


===== AVERAGE METRICS =====
ROUGE-1 F1:   0.2885
ROUGE-2 F1:   0.0862
ROUGE-L F1:   0.2451
BERTScore F1:0.8901


In [ ]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.14.0
    Uninstalling gradio_client-1.14.0:
      Successfully uninstalled gradio_client-1.14.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.50.0
    Uninstalling gradio-5.50.0:
      Successfully uninstalled gradio-5.50.0


In [ ]:
import torch
from datasets import load_dataset
from rouge_score import rouge_scorer
from bert_score import score
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# --------------------------------------------
# Config
# --------------------------------------------
MODEL_DIR = "/content/mistral-slm-summary"
EXAMPLE_ID = 25
SUMMARY_LENGTH = "short"

device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------------------------
# Load model & tokenizer (SAFE)
# --------------------------------------------
model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.float16,
    device_map={"": 0},        # ✅ force everything on GPU
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

model.eval()
model.config.use_cache = True

# --------------------------------------------
# Summarization function
# --------------------------------------------
def summarize(article, length):
    if length == "short":
        instruction = "Summarize the article in 2-3 sentences."
        max_tokens = 80
    elif length == "medium":
        instruction = "Summarize the article in 3-4 sentences."
        max_tokens = 120
    else:
        instruction = "Summarize the article in 5-6 sentences."
        max_tokens = 180

    prompt = f"""### Instruction:
{instruction}

### Article:
{article}

### Summary:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("### Summary:")[-1].strip()

# --------------------------------------------
# Load dataset example
# --------------------------------------------
dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation")

article = dataset[EXAMPLE_ID]["article"]
reference = dataset[EXAMPLE_ID]["highlights"]

# --------------------------------------------
# Run inference
# --------------------------------------------
prediction = summarize(article, SUMMARY_LENGTH)

# --------------------------------------------
# Print outputs
# --------------------------------------------
print("=" * 100)
print("ARTICLE (truncated):\n")
print(article[:900] + "...\n")
print("-" * 100)
print("REFERENCE SUMMARY:\n")
print(reference + "\n")
print("-" * 100)
print("MODEL SUMMARY:\n")
print(prediction)
print("=" * 100)

# --------------------------------------------
# ROUGE evaluation
# --------------------------------------------
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeLsum"],
    use_stemmer=True
)

rouge_scores = scorer.score(reference, prediction)

print("\nROUGE SCORES (single example):")
for k, v in rouge_scores.items():
    print(f"{k.upper():10s} | P: {v.precision:.4f} | R: {v.recall:.4f} | F1: {v.fmeasure:.4f}")

# --------------------------------------------
# BERTScore evaluation
# --------------------------------------------
P, R, F1 = score(
    [prediction],
    [reference],
    lang="en",
    verbose=False
)

print("\nBERTScore:")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")



ARTICLE (truncated):

(Rolling Stone)Kanye West fans are used to seeing thousand-dollar price tags attached to the rapper's Air Yeezy sneakers when they're put up for sale on eBay, but the auction site is now being overrun by sellers offering up plastic bags full of air allegedly captured at West's concerts. The gag started Friday when one ambitious seller attempted to sell a Zipperseal bag with "Air From Kanye Show," with 90 bidders driving the price tag to over $60,000, the Telegraph reports. While that auction was ultimately pulled from the auction site, eBay has since been flooded with similar auctions promising air bagged from a Yeezus show, even though the copycats lacked certificates of authenticity proving the air -- likely carbon dioxide -- was grabbed at a West concert. eBay has apparently stopped trying to swat down the new "bagged air" auctions, so buyer beware at this point. Rolling Stone: Kanye ...

-------------------------------------------------------------------------

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore:
Precision: 0.8736
Recall:    0.8795
F1:        0.8766


In [ ]:
# ============================
# GRADIO + COLAB HARD FIX
# ============================
import os
os.environ["GRADIO_ANALYTICS_ENABLED"] = "False"
os.environ["GRADIO_NOTEBOOK"] = "False"

import gradio as gr
import torch
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# ----------------------------
# Load your trained PEFT model
# ----------------------------
PEFT_MODEL_DIR = "/content/mistral-slm-summary"

model = AutoPeftModelForCausalLM.from_pretrained(
    PEFT_MODEL_DIR,
    torch_dtype=torch.float16,
    device_map={"": 0},   # FORCE GPU
)
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

model.eval()
model.config.use_cache = True

# ----------------------------
# Summarization function
# ----------------------------
def summarize(article, length):
    if length == "short":
        instruction = "Summarize the article in 2-3 sentences."
        max_tokens = 80
    elif length == "medium":
        instruction = "Summarize the article in 3-4 sentences."
        max_tokens = 130
    else:
        instruction = "Summarize the article in 5-6 sentences."
        max_tokens = 200

    prompt = f"""### Instruction:
{instruction}

### Article:
{article}

### Summary:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            repetition_penalty=1.3,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("### Summary:")[-1].strip()

# ----------------------------
# Gradio UI (Blocks = safer)
# ----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 📝 Mistral 7B News Summarizer")

    article = gr.Textbox(
        lines=12,
        label="Article",
        placeholder="Paste your article here..."
    )

    length = gr.Radio(
        ["short", "medium", "long"],
        value="short",
        label="Summary Length"
    )

    output = gr.Textbox(label="Generated Summary")

    btn = gr.Button("Summarize")
    btn.click(summarize, inputs=[article, length], outputs=output)

# ----------------------------
# LAUNCH (THIS IS THE KEY)
# ----------------------------
demo.launch()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://838a8bb2b387c2767d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
